<a href="https://colab.research.google.com/github/jsmazorra/DS-Unit-2-Kaggle-Challenge/blob/master/module3-cross-validation/Johan_Mazorra_LS_DS13_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [20]:
train.describe()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,year_recorded,month_recorded,day_recorded,years
count,47520.000000,47520.000000,46078.000000,46078.000000,47520.000000,47520.000000,47520.000000,47520.000000,47520.000000,47520.000000,47520.000000,47520.000000,47520.000000
mean,321.925261,669.567656,35.149033,-5.884512,0.477736,15.258291,5.616751,179.805513,1302.768939,2011.921907,4.366604,15.625926,709.152967
std,3197.240487,693.005745,2.604241,2.805599,13.312977,17.530228,9.621230,463.081564,950.955437,0.959690,3.025041,8.685898,950.617560
min,0.000000,-63.000000,29.607122,-11.649440,0.000000,1.000000,0.000000,0.000000,0.000000,2002.000000,1.000000,1.000000,-7.000000
25%,0.000000,0.000000,33.284679,-8.633876,0.000000,5.000000,2.000000,0.000000,0.000000,2011.000000,2.000000,8.000000,8.000000
50%,0.000000,372.500000,35.008578,-5.170151,0.000000,12.000000,3.000000,25.000000,1986.000000,2012.000000,3.000000,16.000000,26.000000
75%,25.000000,1320.000000,37.223501,-3.375068,0.000000,17.000000,5.000000,215.000000,2004.000000,2013.000000,7.000000,23.000000,2011.000000
max,350000.000000,2770.000000,40.344301,-0.998464,1776.000000,99.000000,80.000000,15300.000000,2013.000000,2013.000000,12.000000,31.000000,2013.000000


In [21]:
test.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,14358.000000,14358.000000,14358.0000,14358.000000,1.435800e+04,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000
mean,37232.859799,324.219996,653.6363,34.082414,-5.697584e+00,0.408971,15.156359,5.713052,187.055439,1298.251985
std,21382.890432,2533.367778,688.2721,6.564449,2.947444e+00,8.231859,17.387588,9.794304,476.065978,952.551852
min,10.000000,0.000000,-57.0000,0.000000,-1.156459e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18765.500000,0.000000,0.0000,33.062317,-8.453125e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37442.000000,0.000000,346.0000,34.898976,-5.087905e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55909.250000,25.000000,1306.0000,37.221606,-3.314240e+00,0.000000,17.000000,5.000000,230.000000,2004.000000
max,74249.000000,200000.000000,2777.0000,40.325016,-2.000000e-08,669.000000,99.000000,80.000000,11469.000000,2013.000000


In [22]:
train.describe(include='object')

,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
count,44616,44603,47520,47520,47234,47520,47520,47520,44876,44392,24988,45077,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520
unique,1716,1929,30661,9,17231,21,124,2082,2,12,2563,2,18,13,7,12,5,7,8,6,5,10,7,3,7,6,3
top,Government Of Tanzania,DWE,none,Lake Victoria,Shuleni,Iringa,Njombe,Igosi,True,VWC,K,True,gravity,gravity,gravity,vwc,user-group,never pay,soft,good,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
freq,7321,13978,2879,8137,420,4250,2003,257,40838,29470,548,31071,21448,21448,21448,32449,42027,20287,40598,40598,26567,13620,13620,36638,22778,27642,25807


In [23]:
test.describe(include='object')

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,recorded_by,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
count,14358,13575,13570,14358,14358,14264,14358,14358,14358,13573,14358,13419,7519,13695,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358
unique,331,960,1075,10615,9,8253,21,124,1934,2,1,11,1772,2,17,13,7,12,5,7,7,8,6,5,5,10,7,3,7,6
top,2011-03-16,Government Of Tanzania,DWE,none,Lake Victoria,Shuleni,Shinyanga,Njombe,Igosi,True,GeoData Consultants Ltd,VWC,Borehole,True,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,communal standpipe,communal standpipe
freq,137,2117,4162,822,2535,136,1258,611,79,12308,14358,8807,158,9442,6168,6168,6168,9780,12639,6098,6098,12237,12237,7997,7997,4211,4211,11127,6790,8260


In [0]:
import numpy as np
from sklearn.model_selection import train_test_split

# Again, let's start by splitting the training data into both train and val.
train, val = train_test_split(train, train_size=.8, test_size=.2, stratify=train['status_group'], random_state=42)

# And let's clean up the data to get the best results.
def fix_data(X):
  X = X.copy() 

  # This is to clean up the zeros, I also added longitude for better results.
  X['latitude'] = X['latitude'].replace(-2e-08,0)
  X['longitude'] = X['longitude'].replace(-2e-08,0)
  cols = ['longitude','latitude']
  for col in cols:
    X[col] = X[col].replace(0, np.nan)
    X[col+'_MISSING']= X[col].isnull()
  
  # Let's drop the columns that are duplicated.
  duplicates = ['quantity_group','payment_type']
  X = X.drop(columns=duplicates)

  X = X.drop(columns=['recorded_by','id'])

  X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
  X['year_recorded'] = X['date_recorded'].dt.year
  X['month_recorded'] = X['date_recorded'].dt.month
  X['day_recorded'] = X['date_recorded'].dt.day
  X = X.drop(columns='date_recorded')
  X['years'] = X['year_recorded'] - X['construction_year']
  X['years_MISSING'] = X['years'].isnull()

  return X

train = fix_data(train)
val = fix_data(val)
test = fix_data(test)

In [0]:
# Next, let's set up the target and features.
target = 'status_group'

features = train.drop(columns=[target])

# Let's get the numerical features.
num_feats = features.select_dtypes(include='number').replace(-2e-08,0)
num_feats = features.select_dtypes(include='number').columns.tolist()

# And then the low cardinal categorical ones.
cat_feats = features.select_dtypes(exclude='number').nunique()
cat_feats = cat_feats[cat_feats <= 30].index.tolist()

features = num_feats + cat_feats

In [0]:
# Make the respective matrices.
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [0]:
# Import the necessary libraries.

from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
import category_encoders as ce
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# I modified the pipeline for better results.
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(bootstrap=True, max_depth=50, max_features='auto', min_samples_leaf= 1, min_samples_split= 5, n_estimators=800, random_state=42, criterion='entropy')
)

In [10]:
# Let's set up the parameters.
param_distributions = {
    'simpleimputer__strategy': ['mean','median'],
    'randomforestclassifier__max_depth': range(1,20,1),
    'randomforestclassifier__n_estimators': range(10,200,10),
    'randomforestclassifier__min_samples_leaf': range(1,21,1),
    'randomforestclassifier__max_features': uniform(0,1)
}

# This is for a random search.
search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=20,
    cv=5,
    scoring='accuracy',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

search.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   23.3s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   46.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 13.5min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed: 18.1min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed: 20.7min
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed: 22.3min
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed: 23.7min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 24.0min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            fill_value=None,


In [12]:
# Okay, that took way longer than expected. Now let's see the final results.
print('Best Hyperparameters:',search.best_params_)
print('Cross Validation Accuracy:',search.best_score_)

Best Hyperparameters: {'randomforestclassifier__max_depth': 18, 'randomforestclassifier__max_features': 0.7358202736452709, 'randomforestclassifier__min_samples_leaf': 3, 'randomforestclassifier__n_estimators': 70, 'simpleimputer__strategy': 'median'}
Cross Validation Accuracy: 0.8036195286195286


In [16]:
# These results were a bit disappointing.
# Let's get the best approximation.
pipeline = search.best_estimator_
pipeline.fit(X_train, y_train)

# Then let's get the predictions.
y_pred = pipeline.predict(X_test)

test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
kaggle_sub = pd.DataFrame(data=y_pred,index=test['id'],columns=['status_group'])

print(kaggle_sub.shape)
kaggle_sub.head()

(14358, 1)


,status_group
id,
50785,functional
51630,functional
17168,functional
45559,non functional
49871,functional


In [15]:
print(sample_submission.shape)
sample_submission.head()

(14358, 2)


,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,functional
4,49871,functional


In [0]:
kaggle_sub.to_csv('Lambda_Kaggle_Challenge_Johan_Mazorra_.csv')